In [518]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [541]:
df=pd.read_csv('F:\\train_F3WbcTw.csv')
df=df.drop('unique_hash',1)
test=pd.read_csv('F:\\test_tOlRoBf.csv')
test=test.drop('unique_hash',1)
df.head()

,unique_hash,text,drug,sentiment
0,2e180be4c9214c1f5ab51fd8cc32bc80c9f612e0,Autoimmune diseases tend to come in clusters. ...,gilenya,2
1,9eba8f80e7e20f3a2f48685530748fbfa95943e4,I can completely understand why you’d want to ...,gilenya,2
2,fe809672251f6bd0d986e00380f48d047c7e7b76,Interesting that it only targets S1P-1/5 recep...,fingolimod,2
3,bd22104dfa9ec80db4099523e03fae7a52735eb6,"Very interesting, grand merci. Now I wonder wh...",ocrevus,2
4,b227688381f9b25e5b65109dd00f7f895e838249,"Hi everybody, My latest MRI results for Brain ...",gilenya,1


In [520]:
df['sentiment']=df['sentiment'].astype('category')

In [521]:
df['sentiment'].value_counts()

2    3825
1     837
0     617
Name: sentiment, dtype: int64

In [543]:
df['text']=df['text'].str.replace('[^a-zA-Z]',' ')
test['text']=test['text'].str.replace('[^a-zA-Z]',' ')

In [523]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf=TfidfVectorizer(stop_words='english',max_df=0.9,max_features=50,ngram_range=(2,2))
tf.fit(df['text'])
df_tf=pd.DataFrame(tf.transform(df['text']).toarray(),columns=tf.get_feature_names()).add_prefix('tfidf_')
df=df.join(df_tf)

In [524]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf=TfidfVectorizer(stop_words='english',max_df=0.9,max_features=50,ngram_range=(2,2))
tf.fit(test['text'])
test_tf=pd.DataFrame(tf.transform(test['text']).toarray(),columns=tf.get_feature_names()).add_prefix('tfidf_')
test=test.join(test_tf)

In [525]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf=TfidfVectorizer(stop_words='english',max_df=0.9,max_features=50)
tf.fit(df['drug'])
df_tf1=pd.DataFrame(tf.transform(df['drug']).toarray(),columns=tf.get_feature_names()).add_prefix('tfidf_')
df=df.join(df_tf1)

In [526]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf=TfidfVectorizer(stop_words='english',max_df=0.9,max_features=50)
tf.fit(test['drug'])
test_tf1=pd.DataFrame(tf.transform(test['drug']).toarray(),columns=tf.get_feature_names()).add_prefix('tfidf_')
test=test.join(test_tf1)

In [527]:
df=df.drop(['text','drug'],1)

In [528]:
test=test.drop(['text','drug'],1)

In [529]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier
from sklearn.preprocessing import scale

In [530]:
x=df.drop(['sentiment'],1)
y=df['sentiment']

In [531]:
x_train,x_test,y_train,y_test = train_test_split(x,y,stratify=y,random_state=42)

In [532]:
from sklearn.naive_bayes import GaussianNB
nb=GaussianNB()
nb.fit(x_train,y_train)
nb.score(x_test,y_test)

0.7143939393939394

In [533]:
dt=DecisionTreeClassifier(max_depth=5)
dt.fit(x_train,y_train)
dt.score(x_test,y_test)

0.7242424242424242

In [534]:
from sklearn.ensemble import AdaBoostClassifier
ada=AdaBoostClassifier(dt,n_estimators=150)
ada.fit(x_train,y_train)
ada.score(x_test,y_test)

0.7242424242424242

In [535]:
from sklearn.ensemble import GradientBoostingClassifier
gb1=GradientBoostingClassifier(n_estimators=150,subsample=0.9)
gb1.fit(x_train,y_train)
gb1.score(x_test,y_test)

0.7227272727272728

In [537]:
y_test=ada.predict(test)
y_test

array([2, 2, 2, ..., 2, 2, 2], dtype=int64)

In [538]:
sub=pd.read_csv('F:\\sample_submission_i5xnIZD.csv')
sub['sentiment']=y_test

In [539]:
sub['sentiment'].value_counts()

2    2918
1       5
0       1
Name: sentiment, dtype: int64

In [540]:
sub.to_csv('F:\\answer.csv',index=False)

In [ ]:
df